# 전처리 실험

이 노트북에서는 이미지 전처리 기법을 실험합니다.


In [ ]:
# 필요한 라이브러리 import
import os
import cv2
import numpy as np
from PIL import Image
from pathlib import Path
import shutil
from tqdm import tqdm
import json
import re
from sklearn.preprocessing import StandardScaler


## 설정


In [ ]:
# 경로 설정
RAW_DATA_DIR = Path("../data/raw")
PROCESSED_DATA_DIR = Path("../data/processed")

# 전처리 설정
IMAGE_SIZE = (224, 224)  # 이미지 리사이즈 크기
NOISE_REDUCTION = True  # 노이즈 제거 여부
NORMALIZE_COLOR = True  # 색상 정규화 여부

# 데이터셋 세트 설정 (실제 데이터 구조에 맞게 수정)
DATASET_SETS = ["dataset_1", "dataset_2", "dataset_3"]

print(f"원본 데이터 경로: {RAW_DATA_DIR}")
print(f"전처리 데이터 저장 경로: {PROCESSED_DATA_DIR}")


## 전처리 함수 정의


In [ ]:
def remove_noise(image):
    """
    이미지 노이즈 제거 (Non-local Means Denoising)
    
    Args:
        image: numpy 배열 이미지
        
    Returns:
        denoised_image: 노이즈 제거된 이미지
    """
    if len(image.shape) == 3:
        # 컬러 이미지
        denoised = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    else:
        # 그레이스케일 이미지
        denoised = cv2.fastNlMeansDenoising(image, None, 10, 7, 21)
    return denoised


def normalize_color(image):
    """
    색상 정규화 (히스토그램 균등화)
    
    Args:
        image: numpy 배열 이미지
        
    Returns:
        normalized_image: 정규화된 이미지
    """
    if len(image.shape) == 3:
        # 컬러 이미지 - YUV 색공간으로 변환 후 Y 채널만 정규화
        yuv = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        yuv[:, :, 0] = cv2.equalizeHist(yuv[:, :, 0])
        normalized = cv2.cvtColor(yuv, cv2.COLOR_YUV2RGB)
    else:
        # 그레이스케일 이미지
        normalized = cv2.equalizeHist(image)
    return normalized


def clean_filename(filename):
    """
    파일명 정리 (특수문자 제거, 공백 처리)
    
    Args:
        filename: 원본 파일명
        
    Returns:
        cleaned_filename: 정리된 파일명
    """
    # 확장자 분리
    name, ext = os.path.splitext(filename)
    
    # 특수문자 제거 및 공백을 언더스코어로 변경
    cleaned = re.sub(r'[^a-zA-Z0-9가-힣]', '_', name)
    cleaned = re.sub(r'_+', '_', cleaned)  # 연속된 언더스코어 제거
    cleaned = cleaned.strip('_')  # 앞뒤 언더스코어 제거
    
    return f"{cleaned}{ext}"


def get_category_label(filepath):
    """
    파일 경로에서 카테고리 라벨 추출
    
    Args:
        filepath: 파일 경로
        
    Returns:
        category: 카테고리 라벨 (예: 'real', 'fake')
    """
    # 경로에서 카테고리 추출 (예: data/raw/real/image.jpg -> 'real')
    parts = Path(filepath).parts
    
    # 'real' 또는 'fake' 키워드 찾기 (대소문자 구분 없음)
    for part in parts:
        part_lower = part.lower()
        if part_lower in ['real', 'authentic']:
            return 'real'
        elif part_lower in ['fake', 'ai', 'generated']:
            return 'fake'
    
    # 기본값 (파일명이나 경로에서 추론 불가능한 경우)
    return 'unknown'


def preprocess_image(image_path, output_size=IMAGE_SIZE, 
                    remove_noise_flag=NOISE_REDUCTION, 
                    normalize_color_flag=NORMALIZE_COLOR):
    """
    단일 이미지 전처리 함수
    
    Args:
        image_path: 이미지 파일 경로
        output_size: 리사이즈할 크기 (width, height)
        remove_noise_flag: 노이즈 제거 여부
        normalize_color_flag: 색상 정규화 여부
        
    Returns:
        processed_image: 전처리된 PIL Image 객체
    """
    # 이미지 로드
    image = cv2.imread(str(image_path))
    if image is None:
        raise ValueError(f"이미지를 로드할 수 없습니다: {image_path}")
    
    # BGR을 RGB로 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # 노이즈 제거
    if remove_noise_flag:
        image = remove_noise(image)
    
    # 색상 정규화
    if normalize_color_flag:
        image = normalize_color(image)
    
    # 리사이즈
    image = cv2.resize(image, output_size, interpolation=cv2.INTER_LINEAR)
    
    # PIL Image로 변환
    processed_image = Image.fromarray(image)
    
    return processed_image


## 데이터셋별 전처리 실행


In [ ]:
def process_dataset_set(set_name, raw_dir=RAW_DATA_DIR, processed_dir=PROCESSED_DATA_DIR):
    """
    특정 데이터셋 세트를 전처리하여 저장
    
    Args:
        set_name: 데이터셋 세트 이름 (예: 'set1')
        raw_dir: 원본 데이터 디렉토리
        processed_dir: 전처리된 데이터 저장 디렉토리
    """
    # 입력 및 출력 경로 설정
    set_raw_dir = raw_dir / set_name
    set_processed_dir = processed_dir / f"{set_name}_processed"
    
    # 원본 데이터 디렉토리 확인
    if not set_raw_dir.exists():
        print(f"⚠️  경로가 존재하지 않습니다: {set_raw_dir}")
        return
    
    # 출력 디렉토리 생성
    set_processed_dir.mkdir(parents=True, exist_ok=True)
    
    # 카테고리별 통계
    category_stats = {}
    
    # 이미지 파일 찾기
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif'}
    image_files = []
    
    for ext in image_extensions:
        image_files.extend(set_raw_dir.rglob(f"*{ext}"))
        image_files.extend(set_raw_dir.rglob(f"*{ext.upper()}"))
    
    if len(image_files) == 0:
        print(f"⚠️  {set_name}에서 이미지 파일을 찾을 수 없습니다.")
        return
    
    print(f"\n📁 {set_name} 처리 시작: {len(image_files)}개 이미지")
    
    # 이미지 전처리 및 저장
    processed_count = 0
    failed_count = 0
    
    for img_path in tqdm(image_files, desc=f"전처리 중 ({set_name})"):
        try:
            # 카테고리 라벨 추출
            category = get_category_label(img_path)
            
            # 카테고리별 디렉토리 생성
            category_dir = set_processed_dir / category
            category_dir.mkdir(parents=True, exist_ok=True)
            
            # 파일명 정리
            cleaned_filename = clean_filename(img_path.name)
            output_path = category_dir / cleaned_filename
            
            # 중복 파일명 처리
            counter = 1
            original_output_path = output_path
            while output_path.exists():
                name, ext = os.path.splitext(cleaned_filename)
                output_path = category_dir / f"{name}_{counter}{ext}"
                counter += 1
            
            # 이미지 전처리
            processed_image = preprocess_image(img_path)
            
            # 저장
            processed_image.save(output_path, quality=95)
            
            # 통계 업데이트
            if category not in category_stats:
                category_stats[category] = 0
            category_stats[category] += 1
            processed_count += 1
            
        except Exception as e:
            print(f"\n❌ 오류 발생 ({img_path}): {str(e)}")
            failed_count += 1
            continue
    
    # 결과 출력
    print(f"\n✅ {set_name} 전처리 완료!")
    print(f"   - 성공: {processed_count}개")
    print(f"   - 실패: {failed_count}개")
    print(f"   - 카테고리별 통계:")
    for cat, count in category_stats.items():
        print(f"     * {cat}: {count}개")
    
    # 메타데이터 저장
    metadata = {
        "set_name": set_name,
        "total_images": len(image_files),
        "processed": processed_count,
        "failed": failed_count,
        "category_stats": category_stats,
        "preprocessing_config": {
            "image_size": IMAGE_SIZE,
            "noise_removal": NOISE_REDUCTION,
            "color_normalization": NORMALIZE_COLOR
        }
    }
    
    metadata_path = set_processed_dir / "metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    print(f"   - 메타데이터 저장: {metadata_path}")


## 전체 데이터셋 전처리 실행


In [ ]:
# 모든 데이터셋 세트 전처리 실행
print("=" * 60)
print("전처리 파이프라인 시작")
print("=" * 60)

for set_name in DATASET_SETS:
    process_dataset_set(set_name)
    print("-" * 60)

print("\n🎉 모든 데이터셋 전처리 완료!")
print(f"전처리된 데이터 저장 위치: {PROCESSED_DATA_DIR}")


## 전처리 결과 확인


In [ ]:
# 전처리된 데이터 구조 확인
print("\n📊 전처리된 데이터 구조:")
print("=" * 60)

for set_name in DATASET_SETS:
    set_processed_dir = PROCESSED_DATA_DIR / f"{set_name}_processed"
    if set_processed_dir.exists():
        print(f"\n{set_name}_processed/")
        for category_dir in sorted(set_processed_dir.iterdir()):
            if category_dir.is_dir():
                image_count = len(list(category_dir.glob("*.jpg"))) + \
                             len(list(category_dir.glob("*.png"))) + \
                             len(list(category_dir.glob("*.jpeg")))
                print(f"  ├── {category_dir.name}/ ({image_count}개 이미지)")
        
        # 메타데이터 확인
        metadata_path = set_processed_dir / "metadata.json"
        if metadata_path.exists():
            with open(metadata_path, 'r', encoding='utf-8') as f:
                metadata = json.load(f)
            print(f"  └── metadata.json (총 {metadata['processed']}개 처리됨)")
    else:
        print(f"\n{set_name}_processed/ (존재하지 않음)")


## 샘플 이미지 시각화 (전처리 전/후 비교)


In [ ]:
import matplotlib.pyplot as plt

def visualize_preprocessing_comparison(raw_path, processed_path, set_name="sample"):
    """
    전처리 전/후 이미지 비교 시각화
    
    Args:
        raw_path: 원본 이미지 경로
        processed_path: 전처리된 이미지 경로
        set_name: 데이터셋 세트 이름
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # 원본 이미지
    raw_img = Image.open(raw_path)
    axes[0].imshow(raw_img)
    axes[0].set_title(f"원본 이미지\n크기: {raw_img.size}", fontsize=12)
    axes[0].axis('off')
    
    # 전처리된 이미지
    processed_img = Image.open(processed_path)
    axes[1].imshow(processed_img)
    axes[1].set_title(f"전처리된 이미지\n크기: {processed_img.size}", fontsize=12)
    axes[1].axis('off')
    
    plt.suptitle(f"{set_name} - 전처리 전/후 비교", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# 샘플 이미지 찾기 및 시각화
sample_found = False
for set_name in DATASET_SETS:
    set_raw_dir = RAW_DATA_DIR / set_name
    set_processed_dir = PROCESSED_DATA_DIR / f"{set_name}_processed"
    
    if set_raw_dir.exists() and set_processed_dir.exists():
        # 첫 번째 이미지 찾기
        raw_images = list(set_raw_dir.rglob("*.jpg")) + list(set_raw_dir.rglob("*.png"))
        if raw_images:
            raw_path = raw_images[0]
            category = get_category_label(raw_path)
            processed_category_dir = set_processed_dir / category
            
            if processed_category_dir.exists():
                processed_images = list(processed_category_dir.glob("*.jpg")) + \
                                 list(processed_category_dir.glob("*.png"))
                if processed_images:
                    processed_path = processed_images[0]
                    visualize_preprocessing_comparison(raw_path, processed_path, set_name)
                    sample_found = True
                    break

if not sample_found:
    print("⚠️  시각화할 샘플 이미지를 찾을 수 없습니다.")


## 데이터셋 통합 및 분할

전처리된 3개 데이터셋을 하나로 합치고 train/val/test로 분할합니다.


In [ ]:
def merge_datasets(processed_dir=PROCESSED_DATA_DIR, output_dir=None, exclude_unknown=True):
    """
    전처리된 여러 데이터셋을 하나로 통합
    
    Args:
        processed_dir: 전처리된 데이터가 있는 디렉토리
        output_dir: 통합된 데이터를 저장할 디렉토리 (기본값: data/processed/all)
        exclude_unknown: unknown 카테고리 제외 여부
        
    Returns:
        merged_stats: 통합 통계 정보
    """
    if output_dir is None:
        # data/processed/all/ 경로로 설정
        output_dir = processed_dir / "all"
    else:
        output_dir = Path(output_dir)
    
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # 통합 통계
    merged_stats = {}
    total_images = 0
    
    # 모든 전처리된 데이터셋 찾기
    processed_sets = [d for d in processed_dir.iterdir() 
                      if d.is_dir() and d.name.endswith("_processed")]
    
    print(f"📦 {len(processed_sets)}개 데이터셋 통합 시작...")
    print("=" * 60)
    
    for processed_set in processed_sets:
        set_name = processed_set.name
        print(f"\n처리 중: {set_name}")
        
        # 카테고리별로 처리
        for category_dir in processed_set.iterdir():
            if not category_dir.is_dir():
                continue
            
            category = category_dir.name
            
            # unknown 카테고리 제외 옵션
            if exclude_unknown and category == "unknown":
                print(f"  ⏭️  {category} 카테고리 제외")
                continue
            
            # 출력 카테고리 디렉토리 생성
            output_category_dir = output_dir / category
            output_category_dir.mkdir(parents=True, exist_ok=True)
            
            # 이미지 파일 찾기
            image_files = []
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                image_files.extend(category_dir.glob(f"*{ext}"))
                image_files.extend(category_dir.glob(f"*{ext.upper()}"))
            
            # 파일 복사 (중복 방지를 위해 원본 파일명에 데이터셋 이름 추가)
            copied_count = 0
            for img_file in image_files:
                # 파일명에 데이터셋 이름 추가하여 중복 방지
                new_filename = f"{set_name}_{img_file.name}"
                output_path = output_category_dir / new_filename
                
                # 이미 존재하는 경우 스킵 (중복 방지)
                if not output_path.exists():
                    shutil.copy2(img_file, output_path)
                    copied_count += 1
            
            # 통계 업데이트
            if category not in merged_stats:
                merged_stats[category] = 0
            merged_stats[category] += copied_count
            total_images += copied_count
            
            print(f"  ✅ {category}: {copied_count}개 이미지 추가")
    
    print("\n" + "=" * 60)
    print(f"✅ 데이터셋 통합 완료!")
    print(f"   총 이미지 수: {total_images}개")
    print(f"   저장 위치: {output_dir}")
    print(f"   카테고리별 통계:")
    for cat, count in merged_stats.items():
        print(f"     - {cat}: {count}개")
    
    # 메타데이터 저장
    metadata = {
        "total_images": total_images,
        "category_stats": merged_stats,
        "merged_datasets": [d.name for d in processed_sets],
        "exclude_unknown": exclude_unknown
    }
    
    metadata_path = output_dir / "metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    print(f"\n   메타데이터 저장: {metadata_path}")
    
    return merged_stats, output_dir


In [ ]:
def split_merged_dataset(merged_dir, 
                        train_ratio=0.7, 
                        val_ratio=0.15, 
                        test_ratio=0.15,
                        random_state=42,
                        stratify=True):
    """
    통합된 데이터셋을 train/val/test로 분할
    
    Args:
        merged_dir: 통합된 데이터가 있는 디렉토리
        train_ratio: 훈련 세트 비율
        val_ratio: 검증 세트 비율
        test_ratio: 테스트 세트 비율
        random_state: 랜덤 시드
        stratify: 클래스별 비율 유지 여부
        
    Returns:
        split_stats: 분할 통계 정보
    """
    from sklearn.model_selection import train_test_split
    import random
    
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, \
        "비율의 합은 1.0이어야 합니다"
    
    merged_path = Path(merged_dir)
    output_base = merged_path.parent.parent  # data/ 디렉토리
    
    # 출력 디렉토리 생성
    train_dir = output_base / "train"
    val_dir = output_base / "val"
    test_dir = output_base / "test"
    
    split_stats = {
        "train": {},
        "val": {},
        "test": {}
    }
    
    print("\n" + "=" * 60)
    print("📊 데이터셋 분할 시작...")
    print("=" * 60)
    print(f"   비율: Train {train_ratio*100:.0f}% / Val {val_ratio*100:.0f}% / Test {test_ratio*100:.0f}%")
    print(f"   Random State: {random_state}")
    print(f"   Stratify: {stratify}")
    print("-" * 60)
    
    # 클래스별로 분할
    for category_dir in sorted(merged_path.iterdir()):
        if not category_dir.is_dir():
            continue
        
        category = category_dir.name
        
        # 이미지 파일 목록 가져오기
        image_files = []
        for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
            image_files.extend(category_dir.glob(f"*{ext}"))
            image_files.extend(category_dir.glob(f"*{ext.upper()}"))
        
        image_files = sorted(image_files)  # 재현성을 위해 정렬
        total_files = len(image_files)
        
        if total_files == 0:
            print(f"⚠️  {category}: 이미지 파일 없음")
            continue
        
        # 랜덤 시드 설정
        random.seed(random_state)
        np.random.seed(random_state)
        
        # train/val/test로 분할
        # 먼저 train과 나머지로 분할
        train_files, temp_files = train_test_split(
            image_files, 
            test_size=(val_ratio + test_ratio), 
            random_state=random_state,
            shuffle=True
        )
        
        # val과 test로 분할
        val_files, test_files = train_test_split(
            temp_files,
            test_size=test_ratio / (val_ratio + test_ratio),
            random_state=random_state,
            shuffle=True
        )
        
        # 파일 복사
        for split_name, file_list, split_dir in [
            ("train", train_files, train_dir),
            ("val", val_files, val_dir),
            ("test", test_files, test_dir)
        ]:
            output_category_dir = split_dir / category
            output_category_dir.mkdir(parents=True, exist_ok=True)
            
            for file_path in file_list:
                shutil.copy2(file_path, output_category_dir / file_path.name)
            
            split_stats[split_name][category] = len(file_list)
        
        print(f"\n📁 {category}:")
        print(f"   Train: {len(train_files)}개 ({len(train_files)/total_files*100:.1f}%)")
        print(f"   Val:   {len(val_files)}개 ({len(val_files)/total_files*100:.1f}%)")
        print(f"   Test:  {len(test_files)}개 ({len(test_files)/total_files*100:.1f}%)")
    
    # 전체 통계 출력
    print("\n" + "=" * 60)
    print("✅ 데이터셋 분할 완료!")
    print("-" * 60)
    
    total_train = sum(split_stats["train"].values())
    total_val = sum(split_stats["val"].values())
    total_test = sum(split_stats["test"].values())
    total_all = total_train + total_val + total_test
    
    print(f"\n📊 전체 통계:")
    print(f"   Train: {total_train}개 ({total_train/total_all*100:.1f}%)")
    print(f"   Val:   {total_val}개 ({total_val/total_all*100:.1f}%)")
    print(f"   Test:  {total_test}개 ({total_test/total_all*100:.1f}%)")
    print(f"   총계:  {total_all}개")
    
    print(f"\n📂 저장 위치:")
    print(f"   Train: {train_dir}")
    print(f"   Val:   {val_dir}")
    print(f"   Test:  {test_dir}")
    
    # 분할 메타데이터 저장
    split_metadata = {
        "split_ratios": {
            "train": train_ratio,
            "val": val_ratio,
            "test": test_ratio
        },
        "random_state": random_state,
        "stratify": stratify,
        "category_stats": split_stats,
        "total_stats": {
            "train": total_train,
            "val": total_val,
            "test": total_test,
            "total": total_all
        }
    }
    
    metadata_path = output_base / "split_metadata.json"
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(split_metadata, f, indent=2, ensure_ascii=False)
    
    print(f"\n   분할 메타데이터 저장: {metadata_path}")
    
    return split_stats


## 통합 및 분할 실행


In [ ]:
# 1단계: 데이터셋 통합
print("🔄 1단계: 데이터셋 통합")
print("=" * 60)
merged_stats, merged_dir = merge_datasets(
    processed_dir=PROCESSED_DATA_DIR,
    exclude_unknown=True  # unknown 카테고리는 제외
)


In [ ]:
# 2단계: train/val/test 분할
print("\n🔄 2단계: 데이터셋 분할")
split_stats = split_merged_dataset(
    merged_dir=merged_dir,
    train_ratio=0.7,   # 70% 훈련
    val_ratio=0.15,    # 15% 검증
    test_ratio=0.15,   # 15% 테스트
    random_state=42,   # 재현성을 위한 시드
    stratify=True      # 클래스별 비율 유지
)


## 분할 결과 확인


In [ ]:
# 분할된 데이터 구조 확인
print("\n📊 최종 데이터 구조:")
print("=" * 60)

data_base = Path("../data")
for split_name in ["train", "val", "test"]:
    split_dir = data_base / split_name
    if split_dir.exists():
        print(f"\n{split_name}/")
        total = 0
        for category_dir in sorted(split_dir.iterdir()):
            if category_dir.is_dir():
                image_count = len(list(category_dir.glob("*.jpg"))) + \
                             len(list(category_dir.glob("*.png"))) + \
                             len(list(category_dir.glob("*.jpeg")))
                total += image_count
                print(f"  ├── {category_dir.name}/ ({image_count:,}개)")
        print(f"  └── 총계: {total:,}개")
    else:
        print(f"\n{split_name}/ (존재하지 않음)")

# 통합 데이터 확인
all_dir = data_base / "processed" / "all"
if all_dir.exists():
    print(f"\nprocessed/all/")
    total = 0
    for category_dir in sorted(all_dir.iterdir()):
        if category_dir.is_dir():
            image_count = len(list(category_dir.glob("*.jpg"))) + \
                         len(list(category_dir.glob("*.png"))) + \
                         len(list(category_dir.glob("*.jpeg")))
            total += image_count
            print(f"  ├── {category_dir.name}/ ({image_count:,}개)")
    print(f"  └── 총계: {total:,}개")


## 데이터 불균형 확인 및 시각화


In [ ]:
# 데이터 불균형 시각화
import matplotlib.pyplot as plt

# 통합 데이터 통계
categories = list(merged_stats.keys())
counts = list(merged_stats.values())

# 분할별 통계
train_counts = [split_stats["train"].get(cat, 0) for cat in categories]
val_counts = [split_stats["val"].get(cat, 0) for cat in categories]
test_counts = [split_stats["test"].get(cat, 0) for cat in categories]

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. 전체 통합 데이터 분포
axes[0].bar(categories, counts, color=['#3498db', '#e74c3c'])
axes[0].set_title('통합 데이터셋 분포', fontsize=14, fontweight='bold')
axes[0].set_ylabel('이미지 수', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)
for i, (cat, count) in enumerate(zip(categories, counts)):
    axes[0].text(i, count, f'{count:,}', ha='center', va='bottom', fontsize=10)

# 2. Train/Val/Test 분할 분포
x = np.arange(len(categories))
width = 0.25

axes[1].bar(x - width, train_counts, width, label='Train', color='#2ecc71')
axes[1].bar(x, val_counts, width, label='Val', color='#f39c12')
axes[1].bar(x + width, test_counts, width, label='Test', color='#9b59b6')

axes[1].set_title('Train/Val/Test 분할 분포', fontsize=14, fontweight='bold')
axes[1].set_ylabel('이미지 수', fontsize=12)
axes[1].set_xticks(x)
axes[1].set_xticklabels(categories)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

# 값 표시
for i, (cat, train, val, test) in enumerate(zip(categories, train_counts, val_counts, test_counts)):
    axes[1].text(i - width, train, f'{train:,}', ha='center', va='bottom', fontsize=8)
    axes[1].text(i, val, f'{val:,}', ha='center', va='bottom', fontsize=8)
    axes[1].text(i + width, test, f'{test:,}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

# 불균형 비율 계산
if len(counts) >= 2:
    imbalance_ratio = max(counts) / min(counts)
    print(f"\n📈 데이터 불균형 비율: {imbalance_ratio:.2f}:1")
    print(f"   (가장 많은 클래스 / 가장 적은 클래스)")
    
    if imbalance_ratio > 3:
        print("   ⚠️  데이터 불균형이 큽니다. 데이터 증강(data augmentation)을 권장합니다.")
    else:
        print("   ✅ 데이터 불균형이 적절한 수준입니다.")
